In [1]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import print_function
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS


In [4]:
def get_exif(filename):
    image = Image.open(filename)
    image.verify()    #Image verify won't output anything if the image is in the correct format
    exif = image._getexif()
    if exif is not None:
         for key, value in exif.items():
                name = TAGS.get(key, key)
                exif[name] = exif.pop(key)
#          if 'GPSInfo' in exif:
#              for key in exif['GPSInfo'].keys():
#                     name = GPSTAGS.get(key,key)
#                     exif['GPSInfo'][name] = exif['GPSInfo'].pop(key)

    return exif

In [24]:
get_exif('test.jpeg')

{40961: 1,
 40962: 4032,
 40963: 3024,
 41990: 0,
 'Orientation': 6,
 'YCbCrPositioning': 1,
 'ResolutionUnit': 2,
 'ExifOffset': 102,
 'XResolution': (72, 1),
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'FlashPixVersion': b'0100',
 'YResolution': (72, 1),
 'ExifVersion': b'0221'}

In [18]:
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
    
         if tag == 'GPSInfo':
                print(idx)
                print(tag)
                try:
                    print(exif.get("GPSInfo").values())
                except:
                    print('no geotagg available')
#             if idx not in exif:
#                 raise ValueError("No EXIF geotagging found")
# 
#                 for (key, val) in GPSTAGS.items():
#                     if key in exif[idx]:
#                         geotagging[val] = exif[idx][key]

    return geotagging


In [21]:
exif = get_exif('test.jpeg')

In [22]:
get_geotagging(exif)

34853
GPSInfo
no geotagg available


{}

In [23]:
GPSTAGS.items()

dict_items([(0, 'GPSVersionID'), (1, 'GPSLatitudeRef'), (2, 'GPSLatitude'), (3, 'GPSLongitudeRef'), (4, 'GPSLongitude'), (5, 'GPSAltitudeRef'), (6, 'GPSAltitude'), (7, 'GPSTimeStamp'), (8, 'GPSSatellites'), (9, 'GPSStatus'), (10, 'GPSMeasureMode'), (11, 'GPSDOP'), (12, 'GPSSpeedRef'), (13, 'GPSSpeed'), (14, 'GPSTrackRef'), (15, 'GPSTrack'), (16, 'GPSImgDirectionRef'), (17, 'GPSImgDirection'), (18, 'GPSMapDatum'), (19, 'GPSDestLatitudeRef'), (20, 'GPSDestLatitude'), (21, 'GPSDestLongitudeRef'), (22, 'GPSDestLongitude'), (23, 'GPSDestBearingRef'), (24, 'GPSDestBearing'), (25, 'GPSDestDistanceRef'), (26, 'GPSDestDistance'), (27, 'GPSProcessingMethod'), (28, 'GPSAreaInformation'), (29, 'GPSDateStamp'), (30, 'GPSDifferential'), (31, 'GPSHPositioningError')])